<a href="https://colab.research.google.com/github/cevitaufik/Belajar-Pengembangan-Machine-Learning/blob/main/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pengembangan_academy/Chessman-image-dataset.zip \
  -O /tmp/Chessman-image-dataset.zip

--2021-06-07 13:46:17--  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pengembangan_academy/Chessman-image-dataset.zip
Resolving dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)... 52.239.197.36
Connecting to dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)|52.239.197.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60684125 (58M) [application/zip]
Saving to: ‘/tmp/Chessman-image-dataset.zip’

/tmp/Chessman-image 100%[===================>]  57.87M   283KB/s    in 58s     

2021-06-07 13:47:16 (1020 KB/s) - ‘/tmp/Chessman-image-dataset.zip’ saved [60684125/60684125]



In [2]:
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
local_zip = '/tmp/Chessman-image-dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()
train_dir = os.path.join('/tmp/Chessman-image-dataset/Chess')
train_datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode = 'nearest',
    validation_split=0.1) # set validation split

In [3]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=8,
    class_mode='categorical',
    subset='training') # set as training data
validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(150, 150),
    batch_size=16,
    class_mode='categorical',
    subset='validation')

Found 499 images belonging to 6 classes.
Found 52 images belonging to 6 classes.


In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import ResNet152V2
model = tf.keras.models.Sequential([
    ResNet152V2(weights="imagenet", include_top=False, input_tensor=Input(shape=(150, 150, 3))),
    # tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')  
])
model.layers[0].trainable = False

234553344/234545216 [==============================] - 3s 0us/step


In [5]:
model.compile(optimizer=tf.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

In [6]:
history = model.fit(train_generator,
                              validation_data=validation_generator,
                              epochs=50,
                              verbose=2)

Epoch 1/50
63/63 - 153s - loss: 9.2712 - accuracy: 0.4329 - val_loss: 3.0828 - val_accuracy: 0.6154
Epoch 2/50
63/63 - 158s - loss: 1.9962 - accuracy: 0.6613 - val_loss: 2.6587 - val_accuracy: 0.6154
Epoch 3/50
63/63 - 173s - loss: 1.2647 - accuracy: 0.7535 - val_loss: 2.1564 - val_accuracy: 0.6346
Epoch 4/50
63/63 - 143s - loss: 1.2848 - accuracy: 0.7735 - val_loss: 1.9646 - val_accuracy: 0.7308
Epoch 5/50
63/63 - 130s - loss: 0.8215 - accuracy: 0.8337 - val_loss: 2.6066 - val_accuracy: 0.5962
Epoch 6/50
63/63 - 125s - loss: 0.9376 - accuracy: 0.7956 - val_loss: 1.3989 - val_accuracy: 0.6538
Epoch 7/50
63/63 - 125s - loss: 0.7508 - accuracy: 0.8357 - val_loss: 1.6344 - val_accuracy: 0.6923
Epoch 8/50
63/63 - 126s - loss: 0.5909 - accuracy: 0.8537 - val_loss: 1.2650 - val_accuracy: 0.7115
Epoch 9/50
63/63 - 126s - loss: 0.5666 - accuracy: 0.8758 - val_loss: 1.2238 - val_accuracy: 0.7500
Epoch 10/50
63/63 - 125s - loss: 0.4521 - accuracy: 0.8938 - val_loss: 1.5366 - val_accuracy: 0.6538